In [369]:
import pandas as pd

In [370]:
asdiv_a = pd.concat([
    pd.read_csv(f"../data/asdiv/fold{i}.txt", names=["id"])
    for i in range(5)
])
asdiv_a = set(asdiv_a["id"].tolist())

In [371]:
df = pd.read_xml('../data/asdiv/asdiv.xml')
df = df[df["ID"].isin(asdiv_a)]
df

,ID,Grade,Source,Body,Question,Solution-Type,Answer,Formula,Class
0,nluds-0001,1,http://www.k5learning.com,Seven red apples and two green apples are in t...,How many apples are in the basket?,Addition,9 (apples),7+2=9,None
1,nluds-0002,1,http://www.k5learning.com,Ellen has six more balls than Marin. Marin has...,How many balls does Ellen have?,Addition,15 (balls),6+9=15,None
2,nluds-0003,1,http://www.k5learning.com,Janet has nine oranges and Sharon has seven or...,How many oranges do Janet and Sharon have toge...,Addition,16 (oranges),9+7=16,None
3,nluds-0004,1,http://www.k5learning.com,Allan brought two balloons and Jake brought fo...,How many balloons did Allan and Jake have in t...,Addition,6 (balloons),2+4=6,None
4,nluds-0005,1,http://www.k5learning.com,Adam has five more apples than Jackie. Jackie ...,How many apples does Adam have?,Addition,14 (apples),5+9=14,None
...,...,...,...,...,...,...,...,...,...
2300,nluds-2301,4,http://www.dadsworksheets.com,There are 3300 bananas in Janice's banana coll...,how big is each group?,Common-Division,44 (bananas),3300/75=44,None
2301,nluds-2302,4,http://www.dadsworksheets.com,Janice has 316 bottle caps that must be put aw...,how many bottle caps must go in each box?,Common-Division,4 (bottle caps),316/79=4,None
2302,nluds-2303,4,http://www.dadsworksheets.com,Chris is inviting 82 friends to a party. He ha...,How many cookies will each friend get?,Common-Division,22 (cookies),1804/82=22,None
2303,nluds-2304,4,http://www.dadsworksheets.com,Keith has 5530 marbles and 3 pencils. If he sh...,how many marbles does each friend get?,Common-Division,70 (marbles),5530/79=70,None


In [372]:
df["Formula"].str.split("=").apply(len).value_counts()

Formula
2    1218
Name: count, dtype: int64

In [373]:
df[df["Formula"].str.split("=").apply(len) != 2]

,ID,Grade,Source,Body,Question,Solution-Type,Answer,Formula,Class


In [374]:
df["Answer"][~df["Answer"].str.contains("(", regex=False)]

875     77
876     26
961     35
1037    70
1038    47
1253     3
1254     2
1255     2
Name: Answer, dtype: object

In [375]:
df["Class"].dropna()

565     self-defined unit
1039    self-defined unit
2202    self-defined unit
2206    self-defined unit
Name: Class, dtype: object

In [376]:
def merge_body_with_question(body: str, question: str):
    if body.endswith("."):
        return body + " " + question
    return body + ", " + question[0].lower() + question[1:]

df["Question"] = df.apply(lambda row: merge_body_with_question(row["Body"], row["Question"]), axis=1)
del df["Body"]

In [377]:
df

,ID,Grade,Source,Question,Solution-Type,Answer,Formula,Class
0,nluds-0001,1,http://www.k5learning.com,Seven red apples and two green apples are in t...,Addition,9 (apples),7+2=9,None
1,nluds-0002,1,http://www.k5learning.com,Ellen has six more balls than Marin. Marin has...,Addition,15 (balls),6+9=15,None
2,nluds-0003,1,http://www.k5learning.com,Janet has nine oranges and Sharon has seven or...,Addition,16 (oranges),9+7=16,None
3,nluds-0004,1,http://www.k5learning.com,Allan brought two balloons and Jake brought fo...,Addition,6 (balloons),2+4=6,None
4,nluds-0005,1,http://www.k5learning.com,Adam has five more apples than Jackie. Jackie ...,Addition,14 (apples),5+9=14,None
...,...,...,...,...,...,...,...,...
2300,nluds-2301,4,http://www.dadsworksheets.com,There are 3300 bananas in Janice's banana coll...,Common-Division,44 (bananas),3300/75=44,None
2301,nluds-2302,4,http://www.dadsworksheets.com,Janice has 316 bottle caps that must be put aw...,Common-Division,4 (bottle caps),316/79=4,None
2302,nluds-2303,4,http://www.dadsworksheets.com,Chris is inviting 82 friends to a party. He ha...,Common-Division,22 (cookies),1804/82=22,None
2303,nluds-2304,4,http://www.dadsworksheets.com,Keith has 5530 marbles and 3 pencils. If he sh...,Common-Division,70 (marbles),5530/79=70,None


In [378]:
grammar = """
?start: expr

?expr: neg

?atom: num
    | func
    | implicit_mul
    | "(" expr ")"

implicit_mul: num ( "(" expr ")" )+
            | "(" expr ")" ( "(" expr ")" )+

?neg: add
    | "-" neg -> neg
    | "-" add -> neg
?add: sub
    | sub ("+" sub)+ -> add
?sub: mul
    | mul ("-" mul)+ -> sub
?mul: div
    | div ("*" div)+ -> mul
?div: pow
    | pow ("/" pow)+ -> div
?pow: perc
    | perc ("**" perc)+ -> pow
?perc: atom "%" -> perc
     | atom
?func: func_name "(" expr ( "," expr )* ")"
?num: SIGNED_NUMBER

?func_name: CNAME

%import common.SIGNED_NUMBER
%import common.CNAME
%import common.WS
%ignore WS
"""

In [379]:
df["expression"], df["result"] = zip(*df["Formula"].str.split("="))

In [380]:
import gadgets
calc = gadgets.gadget.Calculator()


df["Answer_num"] = df["Answer"].str.split("(").str[0].str.strip()
result_matches = df.apply(lambda row: gadgets.metrics.are_numeric_results_same(row["result"], row["Answer_num"]), axis=1)
result_matches.value_counts()

True     1190
False      28
Name: count, dtype: int64

In [381]:
df[~result_matches]

,ID,Grade,Source,Question,Solution-Type,Answer,Formula,Class,expression,result,Answer_num
753,nluds-0754,4,http://www.ck12.org,A washing machine can wash 28 pounds of clothe...,Ceil-Division,8 (washing machines),200/28=7 r4,None,200/28,7 r4,8
755,nluds-0756,4,http://www.ck12.org,There are 757 pounds of sand to be filled into...,Ceil-Division,12 (bags),757/65=11 r42,None,757/65,11 r42,12
757,nluds-0758,4,http://www.ck12.org,If Simon needs to make 519 gift bags at a rate...,Ceil-Division,13 (days),519/42=12 r15,None,519/42,12 r15,13
759,nluds-0760,4,http://www.ck12.org,Jane can arrange 16 vases of flowers in a day....,Ceil-Division,16 (days),248/16=15 r8,None,248/16,15 r8,16
761,nluds-0762,4,http://www.ck12.org,A box can hold 18 pieces of chalk. If there ar...,Ceil-Division,194 (boxes),3484/18=193 r10,None,3484/18,193 r10,194
763,nluds-0764,4,http://www.ck12.org,A paper bag can hold 16 cookies. If Edgar buys...,Ceil-Division,19 (paper bags),292/16=18 r4,None,292/16,18 r4,19
765,nluds-0766,4,http://www.ck12.org,A bus can transport 48 passengers. If a school...,Ceil-Division,26 (buses),1230/48=25 r30,None,1230/48,25 r30,26
767,nluds-0768,4,http://www.ck12.org,If Tom plans to fix 158 watches at the rate of...,Ceil-Division,14 (days),158/12=13 r2,None,158/12,13 r2,14
771,nluds-0772,4,http://www.ck12.org,The capacity of a tank is 32 gallons. If a com...,Ceil-Division,23 (tanks),728/32=22 r24,None,728/32,22 r24,23
773,nluds-0774,3,http://www.ck12.org,Mrs. Smith gave 53 strawberries to 8 girls equ...,Floor-Division,6 (strawberries),53/8=6 r5,None,53/8,6 r5,6


In [382]:
from typing import Iterable

import warnings
import sympy
import lark

class TreeEvaluator:

    def __init__(self, calc: gadgets.gadget.Calculator, parser: lark.Lark) -> None:
        self.cache = {}
        self.calc = calc
        self.parser = parser

    def eval_tree(self, tree: lark.Tree | lark.Token) -> tuple[str, sympy.Expr]:
        if tree not in self.cache:
            self.cache[tree] = self._eval_tree(tree)
        return self.cache[tree]

    def _eval_tree(self, tree: lark.Tree | lark.Token) -> tuple[str, sympy.Expr]:
        if isinstance(tree, lark.Token):
            if tree.type in ("SIGNED_NUMBER", "NUMBER", "CNAME"):
                return None, self.calc.evaluate(tree.value)
            else:
                raise ValueError(f"unknown token {tree}")
        
        assert isinstance(tree.data, str)
        operation = tree.data
        args_nodes = tree.children
        args = [self._format_arg(operation, self.eval_tree(arg_node)[1]) for arg_node in args_nodes]
        inputs = self._format_op(operation, args)
        return inputs, self.calc.evaluate(inputs)
    
    def _format_op(self, op: str, args: list[str]) -> str:
        if op == "neg":
            assert len(args) == 1
            return "-" + args[0]
        if op == "add" or op == "implicit_add":
            return " + ".join(args)  
        if op == "sub":
            return " - ".join(args)
        if op == "mul" or op == "implicit_mul":
            return " * ".join(args)
        if op == "div":
            return " / ".join(args)
        if op == "pow":
            return " ** ".join(args)
        if op == "perc":
            assert len(args) == 1
            return f"{args[0]} / 100"
        if op == "func":
            assert len(args) >= 1
            return args[0] + "("+ ", ".join(args[1:]) + ")"
        raise ValueError(f"unknown operation {op}")

    def _format_arg(self, parent, value_expr) -> str:
        value_str = self.calc.format_sympy_number(value_expr, add_approx=False)
        if isinstance(value_expr, sympy.core.function.FunctionClass):
            return value_str
        if isinstance(value_expr, sympy.core.function.Application):
            return value_str
        if parent == "func":
            return value_str
        if isinstance(value_expr, (sympy.Float, sympy.Integer, sympy.NumberSymbol)):
            if value_expr < 0:
                return "(" + value_str + ")"
            return value_str
        if isinstance(value_expr, sympy.Rational):
            return "(" + value_str + ")"
        if isinstance(value_expr, (sympy.Mul, sympy.Pow, sympy.Add)):
            return "(" + value_str + ")"
        warnings.warn(f"weird value type {type(value_expr)} for {value_expr} (string: '{value_str}')")
        return "(" + value_str + ")"

    def dfs(self, tree: lark.Tree | lark.Token) -> Iterable[lark.Tree | lark.Token]:
        if isinstance(tree, lark.Tree):
            for child in tree.children:
                yield from self.dfs(child)
        yield tree

    def expr_to_steps(self, expr: str, drop_repeated: bool = True) -> tuple[list[gadgets.datatypes.Interaction], sympy.Expr]:
        tree = self.parser.parse(expr)
        steps = []
        for subtree in self.dfs(tree):
            step = self.eval_tree(subtree)
            if step is None:
                continue
            if step[0] is None:
                continue
            inputs, output_expr = step
            interaction = gadgets.datatypes.Interaction(
                gadget_id="calculator",
                inputs=inputs,
                outputs=self.calc.format_sympy_number(output_expr),
            )
            if drop_repeated and interaction in steps:
                continue
            steps.append(interaction)
            
        _, result = self.eval_tree(tree)
        return steps, result
    

In [383]:
calc = gadgets.gadget.Calculator()

parser = lark.Lark(grammar)
tree_evaluator = TreeEvaluator(calc, parser)

df["chain"], df["evaluated_result"] = zip(*df["expression"].apply(tree_evaluator.expr_to_steps))
df

,ID,Grade,Source,Question,Solution-Type,Answer,Formula,Class,expression,result,Answer_num,chain,evaluated_result
0,nluds-0001,1,http://www.k5learning.com,Seven red apples and two green apples are in t...,Addition,9 (apples),7+2=9,None,7+2,9,9,[gadget_id='calculator' inputs='7 + 2' outputs...,9
1,nluds-0002,1,http://www.k5learning.com,Ellen has six more balls than Marin. Marin has...,Addition,15 (balls),6+9=15,None,6+9,15,15,[gadget_id='calculator' inputs='6 + 9' outputs...,15
2,nluds-0003,1,http://www.k5learning.com,Janet has nine oranges and Sharon has seven or...,Addition,16 (oranges),9+7=16,None,9+7,16,16,[gadget_id='calculator' inputs='9 + 7' outputs...,16
3,nluds-0004,1,http://www.k5learning.com,Allan brought two balloons and Jake brought fo...,Addition,6 (balloons),2+4=6,None,2+4,6,6,[gadget_id='calculator' inputs='2 + 4' outputs...,6
4,nluds-0005,1,http://www.k5learning.com,Adam has five more apples than Jackie. Jackie ...,Addition,14 (apples),5+9=14,None,5+9,14,14,[gadget_id='calculator' inputs='5 + 9' outputs...,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2300,nluds-2301,4,http://www.dadsworksheets.com,There are 3300 bananas in Janice's banana coll...,Common-Division,44 (bananas),3300/75=44,None,3300/75,44,44,[gadget_id='calculator' inputs='3_300 / 75' ou...,44
2301,nluds-2302,4,http://www.dadsworksheets.com,Janice has 316 bottle caps that must be put aw...,Common-Division,4 (bottle caps),316/79=4,None,316/79,4,4,[gadget_id='calculator' inputs='316 / 79' outp...,4
2302,nluds-2303,4,http://www.dadsworksheets.com,Chris is inviting 82 friends to a party. He ha...,Common-Division,22 (cookies),1804/82=22,None,1804/82,22,22,[gadget_id='calculator' inputs='1_804 / 82' ou...,22
2303,nluds-2304,4,http://www.dadsworksheets.com,Keith has 5530 marbles and 3 pencils. If he sh...,Common-Division,70 (marbles),5530/79=70,None,5530/79,70,70,[gadget_id='calculator' inputs='5_530 / 79' ou...,70


In [384]:
import numpy as np

problematic = ~np.isclose(df["evaluated_result"].apply(float), df["Answer_num"].apply(float))
df[problematic]

,ID,Grade,Source,Question,Solution-Type,Answer,Formula,Class,expression,result,Answer_num,chain,evaluated_result
574,nluds-0575,3,http://www.k5learning.com,Johnny practiced for the track team and ran 3 ...,Common-Division,3.333 (minutes),10/3=3.333,None,10/3,3.333,3.333,[gadget_id='calculator' inputs='10 / 3' output...,10/3
753,nluds-0754,4,http://www.ck12.org,A washing machine can wash 28 pounds of clothe...,Ceil-Division,8 (washing machines),200/28=7 r4,None,200/28,7 r4,8,[gadget_id='calculator' inputs='200 / 28' outp...,50/7
755,nluds-0756,4,http://www.ck12.org,There are 757 pounds of sand to be filled into...,Ceil-Division,12 (bags),757/65=11 r42,None,757/65,11 r42,12,[gadget_id='calculator' inputs='757 / 65' outp...,757/65
757,nluds-0758,4,http://www.ck12.org,If Simon needs to make 519 gift bags at a rate...,Ceil-Division,13 (days),519/42=12 r15,None,519/42,12 r15,13,[gadget_id='calculator' inputs='519 / 42' outp...,173/14
759,nluds-0760,4,http://www.ck12.org,Jane can arrange 16 vases of flowers in a day....,Ceil-Division,16 (days),248/16=15 r8,None,248/16,15 r8,16,[gadget_id='calculator' inputs='248 / 16' outp...,31/2
761,nluds-0762,4,http://www.ck12.org,A box can hold 18 pieces of chalk. If there ar...,Ceil-Division,194 (boxes),3484/18=193 r10,None,3484/18,193 r10,194,[gadget_id='calculator' inputs='3_484 / 18' ou...,1742/9
763,nluds-0764,4,http://www.ck12.org,A paper bag can hold 16 cookies. If Edgar buys...,Ceil-Division,19 (paper bags),292/16=18 r4,None,292/16,18 r4,19,[gadget_id='calculator' inputs='292 / 16' outp...,73/4
765,nluds-0766,4,http://www.ck12.org,A bus can transport 48 passengers. If a school...,Ceil-Division,26 (buses),1230/48=25 r30,None,1230/48,25 r30,26,[gadget_id='calculator' inputs='1_230 / 48' ou...,205/8
767,nluds-0768,4,http://www.ck12.org,If Tom plans to fix 158 watches at the rate of...,Ceil-Division,14 (days),158/12=13 r2,None,158/12,13 r2,14,[gadget_id='calculator' inputs='158 / 12' outp...,79/6
771,nluds-0772,4,http://www.ck12.org,The capacity of a tank is 32 gallons. If a com...,Ceil-Division,23 (tanks),728/32=22 r24,None,728/32,22 r24,23,[gadget_id='calculator' inputs='728 / 32' outp...,91/4


In [385]:
import math


equals_with_ceil = np.isclose(df["evaluated_result"].apply(math.ceil).apply(float), df["Answer_num"].apply(float))
equals_with_floor = np.isclose(df["evaluated_result"].apply(math.floor).apply(float), df["Answer_num"].apply(float))

df[problematic & (~equals_with_ceil & ~equals_with_floor)]

,ID,Grade,Source,Question,Solution-Type,Answer,Formula,Class,expression,result,Answer_num,chain,evaluated_result
574,nluds-0575,3,http://www.k5learning.com,Johnny practiced for the track team and ran 3 ...,Common-Division,3.333 (minutes),10/3=3.333,None,10/3,3.333,3.333,[gadget_id='calculator' inputs='10 / 3' output...,10/3


In [386]:
df.loc[problematic & equals_with_ceil, "expression"] = df.loc[problematic & equals_with_ceil, "expression"].apply(lambda x: "ceiling(" + x + ")")
df.loc[problematic & equals_with_floor, "expression"] = df.loc[problematic & equals_with_floor, "expression"].apply(lambda x: "floor(" + x + ")")
df[problematic]

,ID,Grade,Source,Question,Solution-Type,Answer,Formula,Class,expression,result,Answer_num,chain,evaluated_result
574,nluds-0575,3,http://www.k5learning.com,Johnny practiced for the track team and ran 3 ...,Common-Division,3.333 (minutes),10/3=3.333,None,10/3,3.333,3.333,[gadget_id='calculator' inputs='10 / 3' output...,10/3
753,nluds-0754,4,http://www.ck12.org,A washing machine can wash 28 pounds of clothe...,Ceil-Division,8 (washing machines),200/28=7 r4,None,ceiling(200/28),7 r4,8,[gadget_id='calculator' inputs='200 / 28' outp...,50/7
755,nluds-0756,4,http://www.ck12.org,There are 757 pounds of sand to be filled into...,Ceil-Division,12 (bags),757/65=11 r42,None,ceiling(757/65),11 r42,12,[gadget_id='calculator' inputs='757 / 65' outp...,757/65
757,nluds-0758,4,http://www.ck12.org,If Simon needs to make 519 gift bags at a rate...,Ceil-Division,13 (days),519/42=12 r15,None,ceiling(519/42),12 r15,13,[gadget_id='calculator' inputs='519 / 42' outp...,173/14
759,nluds-0760,4,http://www.ck12.org,Jane can arrange 16 vases of flowers in a day....,Ceil-Division,16 (days),248/16=15 r8,None,ceiling(248/16),15 r8,16,[gadget_id='calculator' inputs='248 / 16' outp...,31/2
761,nluds-0762,4,http://www.ck12.org,A box can hold 18 pieces of chalk. If there ar...,Ceil-Division,194 (boxes),3484/18=193 r10,None,ceiling(3484/18),193 r10,194,[gadget_id='calculator' inputs='3_484 / 18' ou...,1742/9
763,nluds-0764,4,http://www.ck12.org,A paper bag can hold 16 cookies. If Edgar buys...,Ceil-Division,19 (paper bags),292/16=18 r4,None,ceiling(292/16),18 r4,19,[gadget_id='calculator' inputs='292 / 16' outp...,73/4
765,nluds-0766,4,http://www.ck12.org,A bus can transport 48 passengers. If a school...,Ceil-Division,26 (buses),1230/48=25 r30,None,ceiling(1230/48),25 r30,26,[gadget_id='calculator' inputs='1_230 / 48' ou...,205/8
767,nluds-0768,4,http://www.ck12.org,If Tom plans to fix 158 watches at the rate of...,Ceil-Division,14 (days),158/12=13 r2,None,ceiling(158/12),13 r2,14,[gadget_id='calculator' inputs='158 / 12' outp...,79/6
771,nluds-0772,4,http://www.ck12.org,The capacity of a tank is 32 gallons. If a com...,Ceil-Division,23 (tanks),728/32=22 r24,None,ceiling(728/32),22 r24,23,[gadget_id='calculator' inputs='728 / 32' outp...,91/4


In [387]:
df["chain"], df["evaluated_result"] = zip(*df["expression"].apply(tree_evaluator.expr_to_steps))

In [388]:
still_problematic = ~np.isclose(df["evaluated_result"].apply(float), df["Answer_num"].apply(float))
df[still_problematic]

,ID,Grade,Source,Question,Solution-Type,Answer,Formula,Class,expression,result,Answer_num,chain,evaluated_result
574,nluds-0575,3,http://www.k5learning.com,Johnny practiced for the track team and ran 3 ...,Common-Division,3.333 (minutes),10/3=3.333,None,10/3,3.333,3.333,[gadget_id='calculator' inputs='10 / 3' output...,10/3


In [389]:
df.loc[574]["Question"]

'Johnny practiced for the track team and ran 3 laps per minute. How many minutes did it take Johnny to run 10 laps?'

In [390]:
df.loc[574, "Answer_num"] = "10/3"

In [391]:
df["result"] = df["Answer_num"]
df["result"]

0        9
1       15
2       16
3        6
4       14
        ..
2300    44
2301     4
2302    22
2303    70
2304    40
Name: result, Length: 1218, dtype: object

In [392]:
df[problematic].iloc[10]["chain"]

[Interaction(gadget_id='calculator', inputs='53 / 8', outputs='53/8 = around 6.625'),
 Interaction(gadget_id='calculator', inputs='floor(53/8)', outputs='6')]

In [393]:
df["chain"] = df.apply(lambda row: gadgets.markup.to_model_markup(chain=row["chain"], result=str(row["result"])), axis=1)
df["chain"]

0              [\n, [7 + 2], \n, [9], \n, \n, [9]]
1            [\n, [6 + 9], \n, [15], \n, \n, [15]]
2            [\n, [9 + 7], \n, [16], \n, \n, [16]]
3              [\n, [2 + 4], \n, [6], \n, \n, [6]]
4            [\n, [5 + 9], \n, [14], \n, \n, [14]]
                           ...                    
2300    [\n, [3_300 / 75], \n, [44], \n, \n, [44]]
2301        [\n, [316 / 79], \n, [4], \n, \n, [4]]
2302    [\n, [1_804 / 82], \n, [22], \n, \n, [22]]
2303    [\n, [5_530 / 79], \n, [70], \n, \n, [70]]
2304    [\n, [1_720 / 43], \n, [40], \n, \n, [40]]
Name: chain, Length: 1218, dtype: object

In [403]:
print(df["chain"].apply(str).str.strip()[problematic].iloc[10])

<gadget id="calculator">53 / 8</gadget>
<output>53/8 = around 6.625</output>

<gadget id="calculator">floor(53/8)</gadget>
<output>6</output>

<result>6</result>


In [404]:
import datasets

datasets.DatasetDict({
    "test": datasets.Dataset.from_pandas(
        pd.DataFrame({
            "id": df["ID"],
            "question": df["Question"],
            "result": df["result"],
            "chain": df["chain"].apply(str).str.strip(),
        }),
        preserve_index=False
    )
}).push_to_hub("MU-NLPC/Calc-asdiv_a")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

In [407]:
datasets.load_dataset("MU-NLPC/Calc-asdiv_a")["test"][1]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/1218 [00:00<?, ? examples/s]

{'id': 'nluds-0002',
 'question': 'Ellen has six more balls than Marin. Marin has nine balls. How many balls does Ellen have?',
 'result': '15',
 'chain': '<gadget id="calculator">6 + 9</gadget>\n<output>15</output>\n\n<result>15</result>'}